<a href="https://colab.research.google.com/github/cds21199/NLP2021/blob/main/Lab2/NLP_Lab3_IgboToEnglish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

## Group Members:
Dino Anastasopoulos (1900661)<br>
Jesse Bristow (1875955)<br>
Chloe Smith (1877342)<br>

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Tue Oct 12 19:26:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ig" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [ ]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/ig-en-baseline


In [ ]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 5.2 MB/s 


In [ ]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ig.xml.gz not found. The following files are available for downloading:

   5 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en-ig.xml.gz
 274 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en.zip
  57 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/ig.zip

 335 MB Total size
./JW300_latest_xml_en-ig.xml.gz ... 100% of 5 MB
./JW300_latest_xml_en.zip ... 100% of 274 MB
./JW300_latest_xml_ig.zip ... 100% of 57 MB


In [ ]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$tgt.en 
print("FIRST WGET SUCCESS")
! mv test.en-$tgt.en test.en
print("FIRST MV SUCCESS")

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$tgt.$tgt 
print("SECOND WGET SUCCESS")
! mv test.en-$tgt.$tgt test.$tgt
print("SECOND MV SUCCESS")

--2021-10-12 19:29:43--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2021-10-12 19:29:44 (13.0 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-10-12 19:29:44--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ig.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 2

In [ ]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [ ]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 4513/471529 lines since contained in test set.


,source_sentence,target_sentence
0,Using Ladders — Do You Make These Safety Checks ?,Iji Ubube Eme Ihe — Ị̀ Na - eme Nnyocha Ndị A ...
1,By Awake ! correspondent in Ireland,Site n’aka onye nta akụkọ Teta ! na Ireland
2,PAUL needed to change a bulb in an outside lig...,Ọ DỊ Paul mkpa ịgbanwe bọlb ọkụ eletrik dị n’i...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [ ]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [ ]:
#start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

#with Pool(cpu_count()-1) as pool:
    #scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
#hours, rem = divmod(time.time() - start_time, 3600)
#minutes, seconds = divmod(rem, 60)
#print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
#df_pp = df_pp.assign(scores=scores)
#df_pp = df_pp[df_pp['scores'] < 95]

In [ ]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
How can we apply this counsel ? — Prov . 15 : 28 ; 16 : 23 .
Consider what Jehovah will do for mankind .
Unscrupulous men slander Jehovah ’ s Witnesses , branding them “ a dangerous cult . ”
Nevertheless , we decided on a code that we would use to warn our spiritual brothers in case of trouble — 4711 , the name of a famous cologne .
“ I Have Found Real Purpose in Life . ” — MARCOS PAULO DE SOUSA
Jesus Saves — How ?
André Caquot , member of the French Institute , speaks of “ the Canaanite cultural substratum at the heart of Israelite religion . ”
Demands from your parents , friends , and teachers ; the physical and emotional changes of puberty ; or the feeling that you are a failure because of some minor shortcoming — all these things can leave you feeling melancholy and sad .
A number of Greeks , perhaps including some who were Jewish proselytes , also believed .
“ Clothe yourselves with the tender affections of compassion , kindness , lowliness of mind , mildness , an



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
# ! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3224 (delta 157), reused 206 (delta 134), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.17 MiB | 12.39 MiB/s, done.
Resolving deltas: 100% (2185/2185), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 5.4 MB/s 
     |████████████████████████████████| 636 kB 17.8 MB/s 
     |████████████████████████████████| 392 kB 71.1 MB/s 
     |████████████████████████████████| 106 kB 69.3 MB/s

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE English Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.ig     test.ig	   train.en
dev.bpe.en	dev.ig	     test.en	     train.bpe.en  train.ig
dev.bpe.ig	test.bpe.en  test.en-any.en  train.bpe.ig
bpe.codes.4000	dev.ig	     test.en-any.en    test.en-any.en.4  train.bpe.ig
dev.bpe.en	test.bpe.en  test.en-any.en.1  test.en-any.en.5  train.en
dev.bpe.ig	test.bpe.ig  test.en-any.en.2  test.ig		 train.ig
dev.en		test.en      test.en-any.en.3  train.bpe.en
BPE English Sentences
Gịnị mere H@@ us@@ ha@@ ị ji kwesị inwe obi ike ka o nwee ike ịkw@@ ado Chineke ?
Gịnị mere anyị ji kwesị inwe obi ike ka anyị nwee ike ịna - akwado Jehova ?
M kpere ekpere ka Jehova nye m obi ike ime ihe m kpebiri .
I@@ we ha ad@@ aj@@ ụ@@ ọla ugbu a . M na - ag@@ azi ahụ ha mgbe niile . ” — Gụọ Ilu 29 : 25 .
[ 1 ] ( par@@ ag@@ ra@@ f nke 7 ) A@@ ha a kpọrọ ụfọdụ ndị n’isiokwu a abụghị ezigbo aha ha .
Combined BPE Vocab
ô
õ@@
◀
×
eazụ
̇
ạ@@
ḥ
▼
ş


In [ ]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.ig	     test.en-any.en    test.en-any.en.4  train.bpe.ig
dev.bpe.en	test.bpe.en  test.en-any.en.1  test.en-any.en.5  train.en
dev.bpe.ig	test.bpe.ig  test.en-any.en.2  test.ig		 train.ig
dev.en		test.en      test.en-any.en.3  train.bpe.en


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 2                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-10-12 20:15:05,169 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-12 20:15:05,185 - INFO - joeynmt.data - Loading training data...
2021-10-12 20:15:13,246 - INFO - joeynmt.data - Building vocabulary...
2021-10-12 20:15:13,538 - INFO - joeynmt.data - Loading dev data...
2021-10-12 20:15:13,593 - INFO - joeynmt.data - Loading test data...
2021-10-12 20:15:13,616 - INFO - joeynmt.data - Data loaded.
2021-10-12 20:15:13,616 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-12 20:15:13,802 - INFO - joeynmt.model - Enc-dec model built.
2021-10-12 20:15:14,847 - INFO - joeynmt.training - Total params: 12188928
2021-10-12 20:15:14,849 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-10-12 20:15:17,919 - INFO - joeynmt.helpers - cfg.name                           : igen_reverse_transformer
2021-10-12 20:15:17,920 - INFO - joeynmt.helpers - cfg.data.src                       : ig
2021-10-12 2

In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!mkdir -p "$gdrive_path/models/${tgt}${src}_reverse_transformer/"
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${tgt}${src}_reverse_transformer/"

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${tgt}${src}_reverse_transformer/validations.txt"

Steps: 1000	Loss: 125674.29688	PPL: 85.25594	bleu: 0.77483	LR: 0.00020000	*
Steps: 2000	Loss: 114127.73438	PPL: 56.66772	bleu: 3.03887	LR: 0.00020000	*
Steps: 3000	Loss: 104987.48438	PPL: 41.01236	bleu: 5.22313	LR: 0.00020000	*
Steps: 4000	Loss: 97724.68750	PPL: 31.72028	bleu: 7.60912	LR: 0.00020000	*
Steps: 5000	Loss: 92512.07812	PPL: 26.37886	bleu: 9.48568	LR: 0.00020000	*
Steps: 6000	Loss: 88613.38281	PPL: 22.98056	bleu: 10.33815	LR: 0.00020000	*
Steps: 7000	Loss: 85255.07812	PPL: 20.40644	bleu: 11.76016	LR: 0.00020000	*
Steps: 8000	Loss: 82558.35938	PPL: 18.54974	bleu: 12.65999	LR: 0.00020000	*
Steps: 9000	Loss: 80236.96875	PPL: 17.08734	bleu: 13.10805	LR: 0.00020000	*
Steps: 10000	Loss: 78100.50781	PPL: 15.84354	bleu: 14.26040	LR: 0.00020000	*
Steps: 11000	Loss: 76242.59375	PPL: 14.83574	bleu: 15.82430	LR: 0.00020000	*
Steps: 12000	Loss: 74420.40625	PPL: 13.90961	bleu: 16.06350	LR: 0.00020000	*
Steps: 13000	Loss: 72893.91406	PPL: 13.17843	bleu: 16.85133	LR: 0.00020000	*
Steps: 140

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${tgt}${src}_reverse_transformer/config.yaml"

2021-10-12 21:03:33,945 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-12 21:03:33,946 - INFO - joeynmt.data - Building vocabulary...
2021-10-12 21:03:34,235 - INFO - joeynmt.data - Loading dev data...
2021-10-12 21:03:34,245 - INFO - joeynmt.data - Loading test data...
2021-10-12 21:03:34,299 - INFO - joeynmt.data - Data loaded.
2021-10-12 21:03:34,315 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-10-12 21:03:34,315 - INFO - joeynmt.prediction - Loading model from models/igen_reverse_transformer/best.ckpt
2021-10-12 21:03:37,176 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-12 21:03:37,361 - INFO - joeynmt.model - Enc-dec model built.
2021-10-12 21:03:37,425 - INFO - joeynmt.prediction - Decoding on dev set (data/igen/dev.bpe.en)...
2021-10-12 21:04:19,299 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-12 21:04:19,299 - WARNING - sacrebleu - It looks like you